# FullNet.ipynb

This file was used to train the single model that used both the TrashNet and the Kaggle data. This was used as a point of comparison for our Two-CNN method.

In [0]:
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models

import numpy as np

from google.colab import drive
drive.mount('/content/drive/')

# HYPERPARAMETERS
train_batch_size = 64
test_batch_size = 64
train_validation_split = 4/5
log_interval = 20
lr = 0.01
momentum = 0.9
num_epochs = 5

def prepare_data():
    training_path = '/content/drive/My Drive/Colab Notebooks/Data/TrashNet_Data/'
    # use the RandomCrop to make all images the same size
    # this gave much better results than transforms.Resize()
    trainval_set = torchvision.datasets.ImageFolder(root=training_path, transform=transforms.Compose([transforms.RandomCrop([256,256],pad_if_needed=True), transforms.ToTensor()]))
    trainval_size = len(trainval_set)
    train_size = int(trainval_size * train_validation_split)

    train_set, val_set = torch.utils.data.random_split(trainval_set, [train_size, trainval_size-train_size])

    train_loader = torch.utils.data.DataLoader(train_set,batch_size=train_batch_size,num_workers=0,shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_set,batch_size=test_batch_size,num_workers=0,shuffle=True)

    return train_set, val_set, train_loader, val_loader

def train(model, criterion, train_loader, optimizer, device):
    model.train() # set the mode of the model to training
    total_loss = 0
    for i, data in enumerate(train_loader):
        imgs, lbls = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, lbls)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        if (i+1) % log_interval == 0:
            mean_loss = total_loss / log_interval
            print('\tbatch {:4d}: loss={:.3f}'.format(i+1, mean_loss))
            total_loss = 0.

def test(model, test_loader, device):
    model.eval()
    total, correct = 0, 0
    T0, F0, T1, F1 = 0, 0, 0, 0
    with torch.no_grad():
        for data in test_loader:
            imgs, lbls = data[0].to(device), data[1].to(device)
            outputs = model(imgs)
            _, preds = torch.max(outputs.data, 1)
            total += lbls.shape[0]
            correct += (preds == lbls).sum().item()

            T0 += (preds[preds == lbls] == 0).sum().item()
            F0 += (preds[preds != lbls] == 0).sum().item()
            T1 += (preds[preds == lbls] == 1).sum().item()
            F1 += (preds[preds != lbls] == 1).sum().item()

    acc = correct / total
    # confusion = np.array([[T0, F0], [F1, T1]])
    print('\tacc={:.3f}'.format(acc))
    # print(confusion)
    

def main():
    device = torch.device('cpu')
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    print('Using {}'.format(device))

    train_set, val_set, train_loader, val_loader = prepare_data()
    print('Training set size: {}'.format(len(train_set)))
    print('Validation set size: {}'.format(len(val_set)))

    model = models.resnet34(pretrained=True)
    # model = torch.load('full_model.pt')
    model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(
        model.parameters(),
        lr = lr,
        momentum = momentum
    )

    for e in range(num_epochs):
        print('Training epoch {}'.format(e))
        train(model, criterion, train_loader, optimizer, device)
        print('Testing on validation set')
        test(model, val_loader, device)

    torch.save(model, 'full_model.pt')



if __name__ == '__main__':
    main()


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
Using cuda:0
Training set size: 12844
Validation set size: 3212
Training epoch 0
	batch   20: loss=1.330
	batch   40: loss=0.232
	batch   60: loss=0.256
	batch   80: loss=0.232
	batch  100: loss=0.218
	batch  120: loss=0.161
	batch  140: loss=0.192
	batch  160: loss=0.162
	batch  180: loss=0.184
	batch  200: loss=0.156
Testing on validation set
	acc=0.935
Training epoch 1
	batch   20: loss=0.142
	batch   40: loss=0.151
	batch   60: loss=0.162
	batch   80: loss=0.149
	batch  100: loss=0.116
	batch  120: loss=0.115
	batch  140: loss=0.126
	batch  160: loss=0.115
	batch  180: loss=0.123
	batch  200: loss=0.105
Testing on validation set
	acc=0.957
Training epoch 2
	batch   20: loss=0.137
	batch   40: loss=0.106
	batch   60: loss=0.106
	batch   80: loss=0.127
	batch  100: loss=0.101
	batch  120: loss=0.080
	batch  140: loss=0.121
	batch  160: loss=0.099
	batch  

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ResNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Conv2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type BatchNorm2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ReLU. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist

In [0]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130
